# Librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten,Dense,Dropout,BatchNormalization,LSTM,Conv1D,MaxPool1D
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

plt.rcParams['figure.figsize'] = [8, 6]

In [ ]:
largo_generacion = 5000000 
bits = 4 
prob_bin = 1/(2 ** bits)

# Data cruda
Se crea el arreglo de muestras partiendo del archivo data_4bits.txt que contiene 5000000 de datos crudos del QRNG en formato de 4 bits que siguen una distribución uniforme.

In [ ]:
muestras = np.zeros(5000000, dtype = 'int8')

f = open("data_4bits.txt", "r")
cadena = f.read()

temp = ''
contador = 0

for a in cadena:
    if a != ",":
        temp += a
    else:
        muestras[contador] = int(temp)
        temp = ''
        contador += 1

Se construye el histograma de probabilidad correspondiente a la distribución uniforme de 4 bits

In [ ]:
probabilidades = np.zeros(2 ** bits)

for i in range(0, len(probabilidades)):
    prob = np.count_nonzero(muestras == i)/len(muestras)
    probabilidades[i] = prob

nums_4bits = np.linspace(0,15,16)

plt.bar(nums_4bits, probabilidades, color = 'blue')
plt.axhline(prob_bin, color = 'red', label = 'y = ' + str(prob_bin))
plt.xlabel('Número de 4 bits')
plt.ylabel('Probabilidad')
plt.legend()
plt.show()

# Conjuntos de ML
Se construyen los conjuntos de entrenamiento y prueba o test. Como input, se utilizan secuencias cuya longitud es dada por la variable "longitud_input". Dichas secuencias se codifican con vectores one-hot. Por otro lado, el corrimiento es dado por "Salto". La variable "porcentaje" determina qué fracción de la muestra será para entrenar y cuanto se utilizará para el test final.

In [ ]:
longitud_input = 32
Salto = 1
porcentaje = 0.50

longitud_entrenamiento = int(porcentaje * largo_generacion / Salto)
longitud_test = int((1 - porcentaje) * largo_generacion / Salto - (longitud_input + 1))

inputs_entrenamiento = np.zeros((longitud_entrenamiento, longitud_input, 2 ** bits), dtype = 'int8')
outputs_entrenamiento = np.zeros((longitud_entrenamiento, 2 ** bits), dtype = 'int8')

inputs_test = np.zeros((longitud_test, longitud_input, 2 ** bits), dtype = 'int8')
outputs_test = np.zeros((longitud_test, 2 ** bits), dtype = 'int8')

for i in range(0,longitud_entrenamiento):
    for j in range(0, longitud_input):
        inputs_entrenamiento[i,j,muestras[i * Salto + j]] = 1
    
    outputs_entrenamiento[i, muestras[i*Salto + longitud_input]] = 1
    
for i in range(longitud_entrenamiento,longitud_entrenamiento + longitud_test):
    for j in range(0, longitud_input):
        inputs_test[i - longitud_entrenamiento, j, muestras[i * Salto + j]] = 1
    
    outputs_test[i - longitud_entrenamiento, muestras[i*Salto + longitud_input]] = 1
    
print(inputs_entrenamiento.shape)
print(outputs_entrenamiento.shape)
print(inputs_test.shape)
print(outputs_test.shape)

# Modelo de red recurrente convolucional (RCNN)
Se construye el modelo de red con keras. 

In [ ]:
input_data_shape = (longitud_input, 2 ** bits)
activacion_conv = 'relu'
activacion_lstm = 'tanh'
activacion_oculta = 'relu'
activacion_output = 'softmax'

RCNN = Sequential()

RCNN.add(Conv1D(input_shape = input_data_shape, filters = 32, kernel_size = 5, activation = activacion_conv,
                padding = 'same', name = 'capa_conv'))
RCNN.add(MaxPool1D(pool_size = 2, name = 'capa_pooling'))
RCNN.add(BatchNormalization())
RCNN.add(Dropout(0.1))
RCNN.add(LSTM(units = 64, activation = activacion_lstm, return_sequences = False, name = 'capa_LSTM'))
RCNN.add(BatchNormalization())
RCNN.add(Dropout(0.1))
RCNN.add(Dense(units = 32, activation = activacion_oculta, name = 'capa_oculta'))
RCNN.add(BatchNormalization())
RCNN.add(Dense(2 ** bits, activation = activacion_output, name = 'capa_outputs'))

RCNN.summary()

# Entrenamiento
Se entrena el modelo de machine learning con el conjunto de entrenamiento. Algunos hiperparámetros como la tasa de entrenamiento y el número de epochs se establecen al principio de la celda. La función de costo es "categorical_crossentropy". Adicionalmente, se utiliza un 10% de la data de entrenamiento como data de validación.

Cabe mencionar que se aplica un EarlyStopping, el cual detiene el entrenamiento cuando el costo de validación no mejora en 10 epochs consecutivos. Al final del entrenamiento, se guarda la red con los parámetros que minimizaron el costo de validación.

In [ ]:
tasa_entrenamiento = 0.0005
numero_epochs = 50
tamanio_minilote = 1024

optimizador = Adam(learning_rate = tasa_entrenamiento)
funcion_costo = 'categorical_crossentropy'
metrica = ['accuracy']
callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights=True)

RCNN.compile(optimizer = optimizador, loss = funcion_costo, metrics = metrica)
entrenamiento = RCNN.fit(inputs_entrenamiento, outputs_entrenamiento, batch_size = tamanio_minilote,
                         callbacks = [callback], validation_split= 0.1, epochs = numero_epochs)

# Gráficas
Se observa la evolución de la red a través de gráficas de costo y precisión tanto sobre el conjunto de entrenamiento como el conjunto de validación.

In [ ]:
plt.plot(entrenamiento.history['accuracy'], label = 'Entrenamiento')
plt.plot(entrenamiento.history['val_accuracy'], label = 'Validación')
plt.axhline(prob_bin, color = 'red', label = 'y = ' + str(prob_bin))
plt.ylabel('Precisión')
plt.xlabel('Epoch')
plt.grid()
plt.legend()
plt.show()

plt.plot(entrenamiento.history['loss'], label = 'Entrenamiento')
plt.plot(entrenamiento.history['val_loss'], label = 'Validación')
plt.ylabel('Costo')
plt.xlabel('Epoch')
plt.grid()
plt.legend()
plt.show()

# Evaluación
Se evalúa la red mediante el conjunto de prueba o test para observar si el modelo pudo generalizarse.

In [ ]:
costo_test, precision_test = RCNN.evaluate(inputs_test, outputs_test)

print("El costo sobre el conjunto de prueba es: " + str(costo_test))
print("La precisión sobre el conjunto de prueba es: " + str(precision_test))

# Prueba estadística
A continuación se preparan los datos necesarios para la prueba estadística. Primero se constuye el arreglo mtp de datos crudos que serán sometidos a los métodos de predicción A y B. El método A es la red neuronal, mientras que el método B consiste en adivinar.

In [ ]:
mtp = np.zeros(2500000, dtype = 'int8')

f = open("prueba_estadistica.txt", "r")
cadena = f.read()

temp = ''
contador = 0

for a in cadena:
    if a != ",":
        temp += a
    else:
        mtp[contador] = int(temp)
        temp = ''
        contador += 1

A partir de mtp se crean 500 subconjuntos. Todos cuentan con 4000 muestras. Se observa cuantos aciertos se logran en cada subconjunto empleando los métodos A y B descritos anteriormente. Por lo tanto, al terminar se tendrán 500 datos emparejados sobre el desempeño de los métodos.

In [ ]:
import random as rd
from tqdm import tqdm

tamanio_subconjunto = 4000
numero_subconjuntos = 500

muestra_A = np.zeros(numero_subconjuntos)
muestra_B = np.zeros(numero_subconjuntos)

longitud_input = 32

for i in tqdm(range(0, numero_subconjuntos)):
    inputs_estadistico = np.zeros((tamanio_subconjunto, longitud_input, 2 ** bits), dtype = 'int8')
    outputs_estadistico_mA = np.zeros((tamanio_subconjunto, 2 ** bits), dtype = 'int8')
    outputs_estadistico_mB = np.zeros(tamanio_subconjunto, dtype = 'int8')
    
    for j in range(0,tamanio_subconjunto):
        for m in range(0, longitud_input):
            inputs_estadistico[j,m,mtp[j + i * tamanio_subconjunto + m]] = 1

        outputs_estadistico_mA[j, mtp[j + i * tamanio_subconjunto + longitud_input]] = 1
        outputs_estadistico_mB[j] = mtp[j + i * tamanio_subconjunto + longitud_input]
     
    #prueba red neuronal
    costo_estadistico, precision_estadistico = RCNN.evaluate(inputs_estadistico, outputs_estadistico_mA, verbose = 0)
    aciertos_mA = round(precision_estadistico * tamanio_subconjunto)
    
    #prueba adivinar aleatoriamente
    aciertos_mB = 0
    for k in range(0,tamanio_subconjunto):
        guess = rd.randint(0,15)
        
        if guess == outputs_estadistico_mB[k]:
            aciertos_mB += 1
    
    muestra_A[i] = aciertos_mA
    muestra_B[i] = aciertos_mB

Finalmente, los datos emparejados se exportan a Excel donde se realiza la prueba estadística

In [ ]:
import pandas as pd
import openpyxl

df = pd.DataFrame(list(zip(muestra_A, muestra_B)), columns=['Aciertos NN', 'Aciertos Adivinando'])
print(df)

df.to_excel('prueba_estadistica_RCNN.xlsx', sheet_name='RCNN')